=======================================================================================
PROBLEM A5
Build and train a neural network model using the Sunspots.csv dataset.
Use MAE as the metrics of your neural network model.
We provided code for normalizing the data. Please do not change the code.
Do not use lambda layers in your model.
The dataset used in this problem is downloaded from kaggle.com/robervalt/sunspots
Desired MAE < 0.15 on the normalized dataset.
========================================================================================

In [ ]:
import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.python.keras.callbacks import EarlyStopping

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/sunspots.csv'
urllib.request.urlretrieve(data_url, 'sunspots.csv')

time_step = []
sunspots = []

In [ ]:
with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
    # YOUR CODE HERE
        sunspots.append(float(row[2]))
    # YOUR CODE HERE
        time_step.append(int(row[0]))

In [ ]:
# YOUR CODE HERE
series= np.array(sunspots)

# Normalization Function. DO NOT CHANGE THIS CODE
min=np.min(series)
max=np.max(series)
series -= min
series /= max
time=np.array(time_step)

In [ ]:
# DO NOT CHANGE THIS CODE
split_time = 3000


time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [ ]:
# DO NOT CHANGE THIS CODE
window_size=30
batch_size=32
shuffle_buffer_size=1000

In [ ]:
train_set = windowed_dataset(x_train, window_size=window_size,
                                batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation="relu", padding='causal',
                            input_shape=[None, 1]),
    tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(64, dropout=0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])



model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.fit(train_set, epochs=100)

In [ ]:
model.save("model_A5.h5")

In [3]:
mae = 0.15

value = 0.0379

print("is 0.0379 < 0.15?", value < mae)

is 0.0379 < 0.15? True
